In [4]:
import tensorflow as tf

In [5]:
import seaborn as sns
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [6]:
## import dataset
df = pd.read_csv('./data/exoplanets-clean.csv')

null_col_vals = df.isnull().sum().to_dict()
NUM_ROWS = df.shape[0]

thresholdAmount = 0.3 * NUM_ROWS

dropped = []
for key in null_col_vals:
    if(null_col_vals[key] >= thresholdAmount):
        dropped.append(key)

df = df.drop(columns=dropped, axis=1)
df = df.dropna()
df.head()

,num_stars,single_planet_exosystem,discoverymethod,disc_year,disc_facility,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
30,3,True,Imaging,2015,Gemini Observatory,0,11688.000000,6209.250000,-3287.250000,0.0,...,-0.120300,5.21149,0.023,-0.023,4.537,0.024,-0.024,5.15806,0.002308,-0.002308
32,2,False,Radial Velocity,1996,Lick Observatory,0,14.651600,0.000100,-0.000100,0.0,...,-0.012300,5.95084,0.023,-0.023,4.015,0.036,-0.036,5.72973,0.000852,-0.000852
35,2,False,Radial Velocity,2004,McDonald Observatory,0,0.736547,0.000001,-0.000001,0.0,...,-0.012300,5.95084,0.023,-0.023,4.015,0.036,-0.036,5.72973,0.000852,-0.000852
52,1,False,Transit,2020,Transiting Exoplanet Survey Satellite (TESS),0,8.463080,0.000060,-0.000060,0.0,...,-0.004625,8.81000,0.100,-0.100,4.529,0.020,-0.020,7.84038,0.000731,-0.000731
53,1,False,Transit,2021,Transiting Exoplanet Survey Satellite (TESS),0,18.859690,0.000080,-0.000080,0.0,...,-0.004625,8.81000,0.100,-0.100,4.529,0.020,-0.020,7.84038,0.000731,-0.000731


In [7]:
attr= pd.get_dummies(df.drop(columns=['single_planet_exosystem'],axis=1), drop_first=True)
target = pd.get_dummies(df['single_planet_exosystem'],prefix='ouput')
print(attr.shape)
print(target.shape)
attr.head()
# target.head()

(3082, 101)
(3082, 2)


,num_stars,disc_year,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxlim,pl_rade,...,disc_facility_TrES,disc_facility_Transiting Exoplanet Survey Satellite (TESS),disc_facility_United Kingdom Infrared Telescope,disc_facility_W. M. Keck Observatory,disc_facility_WASP-South,disc_facility_XO,pl_bmassprov_Mass,pl_bmassprov_Msin(i)/sin(i),pl_bmassprov_Msini,st_metratio_[M/H]
30,3,2015,0,11688.000000,6209.250000,-3287.250000,0.0,13.20000,0.0,13.400,...,False,False,False,False,False,False,True,False,False,True
32,2,1996,0,14.651600,0.000100,-0.000100,0.0,0.11340,0.0,13.900,...,False,False,False,False,False,False,True,False,False,False
35,2,2004,0,0.736547,0.000001,-0.000001,0.0,0.01544,0.0,1.875,...,False,False,False,False,False,False,True,False,False,False
52,1,2020,0,8.463080,0.000060,-0.000060,0.0,0.06490,0.0,3.957,...,False,True,False,False,False,False,True,False,False,False
53,1,2021,0,18.859690,0.000080,-0.000080,0.0,0.11080,0.0,2.522,...,False,True,False,False,False,False,True,False,False,False


In [8]:
attr.head()

,num_stars,disc_year,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxlim,pl_rade,...,disc_facility_TrES,disc_facility_Transiting Exoplanet Survey Satellite (TESS),disc_facility_United Kingdom Infrared Telescope,disc_facility_W. M. Keck Observatory,disc_facility_WASP-South,disc_facility_XO,pl_bmassprov_Mass,pl_bmassprov_Msin(i)/sin(i),pl_bmassprov_Msini,st_metratio_[M/H]
30,3,2015,0,11688.000000,6209.250000,-3287.250000,0.0,13.20000,0.0,13.400,...,False,False,False,False,False,False,True,False,False,True
32,2,1996,0,14.651600,0.000100,-0.000100,0.0,0.11340,0.0,13.900,...,False,False,False,False,False,False,True,False,False,False
35,2,2004,0,0.736547,0.000001,-0.000001,0.0,0.01544,0.0,1.875,...,False,False,False,False,False,False,True,False,False,False
52,1,2020,0,8.463080,0.000060,-0.000060,0.0,0.06490,0.0,3.957,...,False,True,False,False,False,False,True,False,False,False
53,1,2021,0,18.859690,0.000080,-0.000080,0.0,0.11080,0.0,2.522,...,False,True,False,False,False,False,True,False,False,False


In [9]:
## convert the data frames to arrays
X=attr.values
Y=target.values

In [10]:
 #creating test and training
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.20,random_state=100)
x_train, x_test, y_train, y_test=train_test_split(X,Y,test_size=.20,random_state=100)
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2465, 101)
(2465, 2)
(617, 101)
(617, 2)


In [11]:
print(x_train.shape[1])
print(y_train.shape)

101
(2465, 2)


In [12]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

dropout_rate = 0.1
epochs = 50
batch_size = 1
learn_rate = 0.001

model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(36, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(24, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(12, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(6, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(y_train.shape[1], activation='softmax'))

adam = Adam(learning_rate=learn_rate)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])



In [28]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file='model_plot1.png',show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [16]:
model.summary

<bound method Model.summary of <Sequential name=sequential, built=True>>

In [29]:
model_history = model.fit(x_train,y_train, batch_size=batch_size,epochs=epochs,validation_split=.2,verbose=1)

Epoch 1/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9285 - loss: 0.1936 - val_accuracy: 0.8641 - val_loss: 0.6415
Epoch 2/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9280 - loss: 0.2090 - val_accuracy: 0.8600 - val_loss: 0.6062
Epoch 3/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9268 - loss: 0.1987 - val_accuracy: 0.8661 - val_loss: 0.4784
Epoch 4/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9255 - loss: 0.1914 - val_accuracy: 0.8661 - val_loss: 0.5330
Epoch 5/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9221 - loss: 0.2082 - val_accuracy: 0.8600 - val_loss: 0.5012
Epoch 6/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9355 - loss: 0.1642 - val_accuracy: 0.8641 - val_loss: 0.5523
Epoch 7/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 976us/step - accuracy: 0.9216 - loss: 0.1902 - val_accuracy: 0.8600 - val_loss: 0.5861
Epoch 8/50
1972/1972 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9218 - loss: 0.2120 

In [30]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
accuracies = model.evaluate(x_test, y_test, verbose=1)
print("Test score:", accuracies[0])
print("Test Accuracy;", accuracies[1])
# give the confusion matrix and classification report
y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(confusion_matrix(y_test_class, y_pred_class))
print(classification_report(y_test_class, y_pred_class))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8240 - loss: 0.7113 
Test score: 0.7868666648864746
Test Accuracy; 0.8168557286262512
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
[[232  49]
 [ 64 272]]
              precision    recall  f1-score   support

           0       0.78      0.83      0.80       281
           1       0.85      0.81      0.83       336

    accuracy                           0.82       617
   macro avg       0.82      0.82      0.82       617
weighted avg       0.82      0.82      0.82       617

